## Pull Financial Historicals

## Goal
Automate pulling financial historicals from North American public companies

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [22]:

headers= {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0'
}


In [23]:

# note that due to anti-scraping techniques, the website is case sensitive, only lower case ticker can access quarterly
ticker = 'titn' 


In [24]:

urls = {}
urls['income annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/"
urls['income quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"
urls['balance sheet annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/"
urls['balance sheet quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/?p=quarterly"
urls['cash flow annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/"
urls['cash flow quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/?p=quarterly"
urls['ratio annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/"
urls['ratio quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"



In [25]:

filename = f'financial statements ({ticker}).xlsx'


In [20]:

with pd.ExcelWriter(filename, engine='xlsxwriter') as xlwriter:

    for key in urls.keys():
        response = requests.get(urls[key], headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        df = pd.read_html(str(soup), attrs={'data-test': 'financials'})[0]

        # drop the last column
        df = df.iloc[:, :-1]

        # Split the DataFrame into the first column and the remaining columns
        first_column = df.iloc[:, 0]
        remaining_columns = df.iloc[:, 1:]
            
        # Reverse the order of the remaining columns
        remaining_columns = remaining_columns.iloc[:, ::-1]
            
        # Concatenate the first column with the reversed remaining columns
        df = pd.concat([first_column, remaining_columns], axis=1)

        df.to_excel(xlwriter, sheet_name=key, index=False)

        worksheet = xlwriter.sheets[key]
        for i, col in enumerate(df.columns):
            max_length = max(df[col].astype(str).map(len).max(), len(col)) + 2  # Adding a little extra space
            worksheet.set_column(i, i, max_length)




ImportError: lxml not found, please install it